# Lab 3.6.1: Custom Tools - SOLUTIONS

**Complete solutions with explanations and alternative approaches**

---

## Setup

In [ ]:
import ast
import operator
import json
import os
import re
from datetime import datetime
from typing import Dict, Any, Optional, List, Union
from dataclasses import dataclass
from enum import Enum

print("Setup complete!")

---

## Exercise 1 Solution: Unit Converter Tool

**Task**: Build a comprehensive unit converter that handles length, weight, and temperature conversions safely.

In [ ]:
class UnitCategory(Enum):
    """Categories of units for conversion."""
    LENGTH = "length"
    WEIGHT = "weight"
    TEMPERATURE = "temperature"
    VOLUME = "volume"
    TIME = "time"


@dataclass
class ConversionResult:
    """Result of a unit conversion."""
    original_value: float
    original_unit: str
    converted_value: float
    target_unit: str
    category: str
    formula: str


class UnitConverter:
    """
    Production-ready unit converter with comprehensive unit support.
    
    Design Decisions:
    1. Uses base units (meters, grams, etc.) as intermediate for conversions
    2. Temperature handled specially due to non-linear conversions
    3. Includes unit aliases ("m" = "meter" = "meters")
    4. Provides conversion formula for transparency
    """
    
    # Conversion factors TO base unit (meter for length, gram for weight, etc.)
    LENGTH_TO_METER = {
        "meter": 1.0, "m": 1.0, "meters": 1.0,
        "kilometer": 1000.0, "km": 1000.0, "kilometers": 1000.0,
        "centimeter": 0.01, "cm": 0.01, "centimeters": 0.01,
        "millimeter": 0.001, "mm": 0.001, "millimeters": 0.001,
        "mile": 1609.344, "mi": 1609.344, "miles": 1609.344,
        "yard": 0.9144, "yd": 0.9144, "yards": 0.9144,
        "foot": 0.3048, "ft": 0.3048, "feet": 0.3048,
        "inch": 0.0254, "in": 0.0254, "inches": 0.0254,
    }
    
    WEIGHT_TO_GRAM = {
        "gram": 1.0, "g": 1.0, "grams": 1.0,
        "kilogram": 1000.0, "kg": 1000.0, "kilograms": 1000.0,
        "milligram": 0.001, "mg": 0.001, "milligrams": 0.001,
        "pound": 453.592, "lb": 453.592, "lbs": 453.592, "pounds": 453.592,
        "ounce": 28.3495, "oz": 28.3495, "ounces": 28.3495,
        "ton": 907185.0, "tons": 907185.0,  # US ton
        "metric_ton": 1000000.0, "tonne": 1000000.0,
    }
    
    VOLUME_TO_LITER = {
        "liter": 1.0, "l": 1.0, "liters": 1.0,
        "milliliter": 0.001, "ml": 0.001, "milliliters": 0.001,
        "gallon": 3.78541, "gal": 3.78541, "gallons": 3.78541,
        "quart": 0.946353, "qt": 0.946353, "quarts": 0.946353,
        "pint": 0.473176, "pt": 0.473176, "pints": 0.473176,
        "cup": 0.236588, "cups": 0.236588,
        "fluid_ounce": 0.0295735, "fl_oz": 0.0295735,
    }
    
    TIME_TO_SECOND = {
        "second": 1.0, "s": 1.0, "sec": 1.0, "seconds": 1.0,
        "minute": 60.0, "min": 60.0, "minutes": 60.0,
        "hour": 3600.0, "h": 3600.0, "hr": 3600.0, "hours": 3600.0,
        "day": 86400.0, "d": 86400.0, "days": 86400.0,
        "week": 604800.0, "wk": 604800.0, "weeks": 604800.0,
        "year": 31536000.0, "yr": 31536000.0, "years": 31536000.0,
    }
    
    TEMPERATURE_UNITS = {"celsius", "c", "fahrenheit", "f", "kelvin", "k"}
    
    def __init__(self):
        """Initialize the unit converter."""
        self.conversion_count = 0
        self.history: List[ConversionResult] = []
    
    def _normalize_unit(self, unit: str) -> str:
        """Normalize unit name to lowercase."""
        return unit.lower().strip().replace(" ", "_")
    
    def _identify_category(self, unit: str) -> Optional[UnitCategory]:
        """Identify which category a unit belongs to."""
        unit = self._normalize_unit(unit)
        
        if unit in self.LENGTH_TO_METER:
            return UnitCategory.LENGTH
        elif unit in self.WEIGHT_TO_GRAM:
            return UnitCategory.WEIGHT
        elif unit in self.VOLUME_TO_LITER:
            return UnitCategory.VOLUME
        elif unit in self.TIME_TO_SECOND:
            return UnitCategory.TIME
        elif unit in self.TEMPERATURE_UNITS:
            return UnitCategory.TEMPERATURE
        return None
    
    def _convert_temperature(self, value: float, from_unit: str, to_unit: str) -> tuple:
        """Handle temperature conversions (non-linear)."""
        from_unit = self._normalize_unit(from_unit)
        to_unit = self._normalize_unit(to_unit)
        
        # Normalize to full names
        unit_map = {"c": "celsius", "f": "fahrenheit", "k": "kelvin"}
        from_unit = unit_map.get(from_unit, from_unit)
        to_unit = unit_map.get(to_unit, to_unit)
        
        # Convert to Celsius first (as intermediate)
        if from_unit == "celsius":
            celsius = value
        elif from_unit == "fahrenheit":
            celsius = (value - 32) * 5/9
        elif from_unit == "kelvin":
            celsius = value - 273.15
        else:
            raise ValueError(f"Unknown temperature unit: {from_unit}")
        
        # Convert from Celsius to target
        if to_unit == "celsius":
            result = celsius
            formula = "direct" if from_unit == "celsius" else f"({value} - 32) * 5/9" if from_unit == "fahrenheit" else f"{value} - 273.15"
        elif to_unit == "fahrenheit":
            result = celsius * 9/5 + 32
            formula = f"({value} * 9/5) + 32" if from_unit == "celsius" else "direct" if from_unit == "fahrenheit" else f"(({value} - 273.15) * 9/5) + 32"
        elif to_unit == "kelvin":
            result = celsius + 273.15
            formula = f"{value} + 273.15" if from_unit == "celsius" else f"(({value} - 32) * 5/9) + 273.15" if from_unit == "fahrenheit" else "direct"
        else:
            raise ValueError(f"Unknown temperature unit: {to_unit}")
        
        return result, formula
    
    def _convert_linear(self, value: float, from_unit: str, to_unit: str, 
                        conversion_table: Dict[str, float]) -> tuple:
        """Handle linear conversions using base unit."""
        from_unit = self._normalize_unit(from_unit)
        to_unit = self._normalize_unit(to_unit)
        
        from_factor = conversion_table[from_unit]
        to_factor = conversion_table[to_unit]
        
        # Convert: value * (from_factor / to_factor)
        result = value * (from_factor / to_factor)
        formula = f"{value} * ({from_factor} / {to_factor})"
        
        return result, formula
    
    def convert(self, value: float, from_unit: str, to_unit: str) -> ConversionResult:
        """
        Convert a value from one unit to another.
        
        Args:
            value: The numeric value to convert
            from_unit: Source unit (e.g., "km", "miles", "celsius")
            to_unit: Target unit
            
        Returns:
            ConversionResult with all details
            
        Raises:
            ValueError: If units are invalid or incompatible
        """
        # Identify categories
        from_category = self._identify_category(from_unit)
        to_category = self._identify_category(to_unit)
        
        if from_category is None:
            raise ValueError(f"Unknown unit: {from_unit}")
        if to_category is None:
            raise ValueError(f"Unknown unit: {to_unit}")
        if from_category != to_category:
            raise ValueError(f"Cannot convert between {from_category.value} and {to_category.value}")
        
        # Perform conversion based on category
        if from_category == UnitCategory.TEMPERATURE:
            result, formula = self._convert_temperature(value, from_unit, to_unit)
        elif from_category == UnitCategory.LENGTH:
            result, formula = self._convert_linear(value, from_unit, to_unit, self.LENGTH_TO_METER)
        elif from_category == UnitCategory.WEIGHT:
            result, formula = self._convert_linear(value, from_unit, to_unit, self.WEIGHT_TO_GRAM)
        elif from_category == UnitCategory.VOLUME:
            result, formula = self._convert_linear(value, from_unit, to_unit, self.VOLUME_TO_LITER)
        elif from_category == UnitCategory.TIME:
            result, formula = self._convert_linear(value, from_unit, to_unit, self.TIME_TO_SECOND)
        
        # Create result
        conversion_result = ConversionResult(
            original_value=value,
            original_unit=from_unit,
            converted_value=round(result, 6),
            target_unit=to_unit,
            category=from_category.value,
            formula=formula
        )
        
        # Track history
        self.conversion_count += 1
        self.history.append(conversion_result)
        
        return conversion_result
    
    def get_supported_units(self, category: Optional[str] = None) -> Dict[str, List[str]]:
        """Get all supported units, optionally filtered by category."""
        all_units = {
            "length": list(set(self.LENGTH_TO_METER.keys())),
            "weight": list(set(self.WEIGHT_TO_GRAM.keys())),
            "volume": list(set(self.VOLUME_TO_LITER.keys())),
            "time": list(set(self.TIME_TO_SECOND.keys())),
            "temperature": list(self.TEMPERATURE_UNITS),
        }
        
        if category:
            return {category: all_units.get(category.lower(), [])}
        return all_units


# Create LangChain-compatible tool
def convert_units(query: str) -> str:
    """
    Convert between units. Query format: 'value from_unit to to_unit'
    Examples: '100 km to miles', '32 fahrenheit to celsius', '5 pounds to kg'
    """
    converter = UnitConverter()
    
    # Parse query with regex
    pattern = r'([\d.]+)\s+(\w+)\s+to\s+(\w+)'
    match = re.match(pattern, query.lower().strip())
    
    if not match:
        return "Invalid format. Use: 'value from_unit to to_unit' (e.g., '100 km to miles')"
    
    try:
        value = float(match.group(1))
        from_unit = match.group(2)
        to_unit = match.group(3)
        
        result = converter.convert(value, from_unit, to_unit)
        return f"{result.original_value} {result.original_unit} = {result.converted_value} {result.target_unit} (Formula: {result.formula})"
    except ValueError as e:
        return f"Error: {str(e)}"


# Test the solution
print("=" * 60)
print("UNIT CONVERTER SOLUTION")
print("=" * 60)

test_conversions = [
    "100 km to miles",
    "32 fahrenheit to celsius",
    "5.5 pounds to kg",
    "1 gallon to liters",
    "24 hours to minutes",
]

for query in test_conversions:
    print(f"Query: {query}")
    print(f"Result: {convert_units(query)}")
    print()

### Alternative Approach: Pint Library

For production systems, consider using the `pint` library which handles unit conversions professionally.

In [ ]:
# Alternative using pint (if installed)
# !pip install pint

try:
    import pint
    ureg = pint.UnitRegistry()
    
    def convert_with_pint(value: float, from_unit: str, to_unit: str) -> str:
        """Convert using the pint library - more robust for production."""
        quantity = value * ureg(from_unit)
        result = quantity.to(to_unit)
        return f"{value} {from_unit} = {result.magnitude:.6f} {to_unit}"
    
    print("Pint library available - more robust for production!")
    print(convert_with_pint(100, "kilometer", "mile"))
except ImportError:
    print("Pint not installed. Our custom solution works great!")

---

## Exercise 2 Solution: Date/Time Calculator

**Task**: Build a tool that calculates date differences, adds/subtracts time, and handles timezones.

In [ ]:
from datetime import datetime, timedelta
from typing import Union
import re


class DateTimeCalculator:
    """
    Safe date/time calculator for agent use.
    
    Supports:
    - Date differences
    - Adding/subtracting time
    - Date parsing in multiple formats
    - Business day calculations
    """
    
    DATE_FORMATS = [
        "%Y-%m-%d",           # 2024-01-15
        "%m/%d/%Y",           # 01/15/2024
        "%d/%m/%Y",           # 15/01/2024
        "%B %d, %Y",          # January 15, 2024
        "%b %d, %Y",          # Jan 15, 2024
        "%Y-%m-%d %H:%M:%S",  # 2024-01-15 14:30:00
    ]
    
    def __init__(self):
        self.operations_log = []
    
    def parse_date(self, date_str: str) -> datetime:
        """Parse date string trying multiple formats."""
        date_str = date_str.strip()
        
        # Handle special keywords
        if date_str.lower() == "today":
            return datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        if date_str.lower() == "tomorrow":
            return datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
        if date_str.lower() == "yesterday":
            return datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)
        
        for fmt in self.DATE_FORMATS:
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
        
        raise ValueError(f"Could not parse date: {date_str}")
    
    def date_difference(self, date1: str, date2: str) -> Dict[str, Any]:
        """Calculate the difference between two dates."""
        d1 = self.parse_date(date1)
        d2 = self.parse_date(date2)
        
        diff = abs(d2 - d1)
        
        # Calculate components
        total_seconds = int(diff.total_seconds())
        days = diff.days
        weeks = days // 7
        years = days // 365
        months = days // 30  # Approximate
        
        hours = (total_seconds % 86400) // 3600
        minutes = (total_seconds % 3600) // 60
        
        result = {
            "from_date": d1.strftime("%Y-%m-%d"),
            "to_date": d2.strftime("%Y-%m-%d"),
            "total_days": days,
            "total_weeks": weeks,
            "total_months": months,
            "total_years": years,
            "breakdown": f"{years} years, {months % 12} months, {days % 30} days",
            "total_hours": days * 24 + hours,
            "total_minutes": days * 24 * 60 + hours * 60 + minutes,
        }
        
        self.operations_log.append(("difference", date1, date2, result))
        return result
    
    def add_time(self, date_str: str, amount: int, unit: str) -> Dict[str, Any]:
        """Add time to a date."""
        date = self.parse_date(date_str)
        unit = unit.lower().rstrip('s')  # Normalize: "days" -> "day"
        
        unit_map = {
            "day": timedelta(days=amount),
            "week": timedelta(weeks=amount),
            "hour": timedelta(hours=amount),
            "minute": timedelta(minutes=amount),
            "second": timedelta(seconds=amount),
            "month": timedelta(days=amount * 30),  # Approximate
            "year": timedelta(days=amount * 365),  # Approximate
        }
        
        if unit not in unit_map:
            raise ValueError(f"Unknown time unit: {unit}")
        
        new_date = date + unit_map[unit]
        
        result = {
            "original_date": date.strftime("%Y-%m-%d %H:%M:%S"),
            "added": f"{amount} {unit}(s)",
            "new_date": new_date.strftime("%Y-%m-%d %H:%M:%S"),
            "day_of_week": new_date.strftime("%A"),
        }
        
        self.operations_log.append(("add", date_str, amount, unit, result))
        return result
    
    def business_days(self, date1: str, date2: str) -> Dict[str, Any]:
        """Calculate business days between two dates (Mon-Fri)."""
        d1 = self.parse_date(date1)
        d2 = self.parse_date(date2)
        
        if d1 > d2:
            d1, d2 = d2, d1
        
        business_days = 0
        current = d1
        
        while current <= d2:
            if current.weekday() < 5:  # Monday = 0, Friday = 4
                business_days += 1
            current += timedelta(days=1)
        
        total_days = (d2 - d1).days + 1
        
        return {
            "from_date": d1.strftime("%Y-%m-%d"),
            "to_date": d2.strftime("%Y-%m-%d"),
            "business_days": business_days,
            "total_days": total_days,
            "weekend_days": total_days - business_days,
        }


def calculate_date(query: str) -> str:
    """
    Date/time calculator tool.
    
    Formats:
    - 'difference 2024-01-01 to 2024-12-31'
    - 'add 30 days to 2024-01-15'
    - 'business days 2024-01-01 to 2024-01-31'
    """
    calc = DateTimeCalculator()
    query = query.lower().strip()
    
    try:
        # Date difference
        diff_match = re.match(r'difference\s+(.+?)\s+to\s+(.+)', query)
        if diff_match:
            result = calc.date_difference(diff_match.group(1), diff_match.group(2))
            return f"Difference: {result['total_days']} days ({result['breakdown']})"
        
        # Add time
        add_match = re.match(r'add\s+(\d+)\s+(\w+)\s+to\s+(.+)', query)
        if add_match:
            result = calc.add_time(add_match.group(3), int(add_match.group(1)), add_match.group(2))
            return f"{result['original_date']} + {result['added']} = {result['new_date']} ({result['day_of_week']})"
        
        # Business days
        biz_match = re.match(r'business days\s+(.+?)\s+to\s+(.+)', query)
        if biz_match:
            result = calc.business_days(biz_match.group(1), biz_match.group(2))
            return f"Business days: {result['business_days']} (out of {result['total_days']} total, {result['weekend_days']} weekend days)"
        
        return "Unknown format. Use: 'difference DATE1 to DATE2', 'add N UNIT to DATE', or 'business days DATE1 to DATE2'"
    
    except Exception as e:
        return f"Error: {str(e)}"


# Test the solution
print("=" * 60)
print("DATE/TIME CALCULATOR SOLUTION")
print("=" * 60)

test_queries = [
    "difference 2024-01-01 to 2024-12-31",
    "add 30 days to today",
    "add 2 weeks to 2024-06-15",
    "business days 2024-01-01 to 2024-01-31",
]

for query in test_queries:
    print(f"Query: {query}")
    print(f"Result: {calculate_date(query)}")
    print()

---

## Exercise 3 Solution: JSON Data Transformer

**Task**: Create a tool that can query, filter, and transform JSON data safely.

In [ ]:
from typing import Any, List, Dict, Optional, Callable
import json
import re


class JSONTransformer:
    """
    Safe JSON data transformer for agent use.
    
    Supports:
    - JSONPath-like queries
    - Filtering with conditions
    - Aggregations (sum, avg, count, min, max)
    - Transformations (map, select fields)
    """
    
    def __init__(self, data: Union[Dict, List, str]):
        """Initialize with JSON data (dict, list, or JSON string)."""
        if isinstance(data, str):
            self.data = json.loads(data)
        else:
            self.data = data
    
    def get_path(self, path: str) -> Any:
        """
        Get value at path. Supports dot notation and array indexing.
        
        Examples:
        - 'users' -> data['users']
        - 'users.0' -> data['users'][0]
        - 'users.0.name' -> data['users'][0]['name']
        - 'users[*].name' -> [u['name'] for u in data['users']]
        """
        if not path or path == '.':
            return self.data
        
        current = self.data
        parts = path.replace('[', '.').replace(']', '').split('.')
        
        for part in parts:
            if not part:
                continue
            
            if part == '*':
                # Wildcard - return all items
                if isinstance(current, list):
                    return current
                elif isinstance(current, dict):
                    return list(current.values())
            
            if isinstance(current, list):
                try:
                    index = int(part)
                    current = current[index]
                except (ValueError, IndexError):
                    # Try to get field from all items
                    current = [item.get(part) for item in current if isinstance(item, dict)]
            elif isinstance(current, dict):
                current = current.get(part)
            else:
                return None
            
            if current is None:
                return None
        
        return current
    
    def filter(self, path: str, condition: str) -> List[Any]:
        """
        Filter array at path by condition.
        
        Condition format: 'field operator value'
        Operators: ==, !=, >, <, >=, <=, contains, startswith, endswith
        
        Example: filter('users', 'age > 25')
        """
        items = self.get_path(path)
        if not isinstance(items, list):
            return []
        
        # Parse condition safely
        operators = {
            '==': lambda a, b: a == b,
            '!=': lambda a, b: a != b,
            '>': lambda a, b: float(a) > float(b),
            '<': lambda a, b: float(a) < float(b),
            '>=': lambda a, b: float(a) >= float(b),
            '<=': lambda a, b: float(a) <= float(b),
            'contains': lambda a, b: str(b).lower() in str(a).lower(),
            'startswith': lambda a, b: str(a).lower().startswith(str(b).lower()),
            'endswith': lambda a, b: str(a).lower().endswith(str(b).lower()),
        }
        
        # Parse: "field operator value"
        pattern = r'(\w+)\s*(==|!=|>=|<=|>|<|contains|startswith|endswith)\s*(.+)'
        match = re.match(pattern, condition.strip())
        
        if not match:
            raise ValueError(f"Invalid condition format: {condition}")
        
        field, op, value = match.groups()
        value = value.strip().strip('"').strip("'")
        
        # Try to convert value to appropriate type
        try:
            if value.lower() == 'true':
                value = True
            elif value.lower() == 'false':
                value = False
            elif '.' in value:
                value = float(value)
            else:
                value = int(value)
        except ValueError:
            pass  # Keep as string
        
        # Apply filter
        op_func = operators[op]
        result = []
        
        for item in items:
            if isinstance(item, dict) and field in item:
                try:
                    if op_func(item[field], value):
                        result.append(item)
                except (TypeError, ValueError):
                    continue
        
        return result
    
    def aggregate(self, path: str, operation: str, field: Optional[str] = None) -> Any:
        """
        Aggregate values at path.
        
        Operations: count, sum, avg, min, max, unique
        
        Example: aggregate('users', 'avg', 'age')
        """
        items = self.get_path(path)
        
        if operation == 'count':
            if isinstance(items, list):
                return len(items)
            return 1 if items else 0
        
        # For other operations, extract field values
        if field and isinstance(items, list):
            values = [item.get(field) for item in items if isinstance(item, dict) and field in item]
        elif isinstance(items, list):
            values = items
        else:
            values = [items]
        
        # Filter to numeric values for numeric operations
        if operation in ('sum', 'avg', 'min', 'max'):
            numeric_values = []
            for v in values:
                try:
                    numeric_values.append(float(v))
                except (TypeError, ValueError):
                    continue
            values = numeric_values
        
        if not values:
            return None
        
        operations = {
            'sum': lambda v: sum(v),
            'avg': lambda v: sum(v) / len(v),
            'min': lambda v: min(v),
            'max': lambda v: max(v),
            'unique': lambda v: list(set(v)),
        }
        
        if operation not in operations:
            raise ValueError(f"Unknown operation: {operation}")
        
        return operations[operation](values)
    
    def select(self, path: str, fields: List[str]) -> List[Dict]:
        """Select specific fields from items at path."""
        items = self.get_path(path)
        if not isinstance(items, list):
            items = [items] if items else []
        
        return [
            {field: item.get(field) for field in fields}
            for item in items
            if isinstance(item, dict)
        ]


def transform_json(data_json: str, query: str) -> str:
    """
    Transform JSON data.
    
    Query formats:
    - 'get PATH' - Get value at path
    - 'filter PATH where CONDITION' - Filter array
    - 'count PATH' - Count items
    - 'sum PATH.FIELD' - Sum field values
    - 'avg PATH.FIELD' - Average field values
    - 'select PATH fields FIELD1,FIELD2' - Select fields
    """
    try:
        transformer = JSONTransformer(data_json)
        query = query.strip().lower()
        
        # Get path
        if query.startswith('get '):
            path = query[4:].strip()
            result = transformer.get_path(path)
            return json.dumps(result, indent=2)
        
        # Filter
        filter_match = re.match(r'filter\s+(\S+)\s+where\s+(.+)', query)
        if filter_match:
            path, condition = filter_match.groups()
            result = transformer.filter(path, condition)
            return json.dumps(result, indent=2)
        
        # Count
        if query.startswith('count '):
            path = query[6:].strip()
            result = transformer.aggregate(path, 'count')
            return str(result)
        
        # Sum/Avg
        for op in ('sum', 'avg', 'min', 'max'):
            if query.startswith(f'{op} '):
                parts = query[len(op)+1:].strip().rsplit('.', 1)
                if len(parts) == 2:
                    path, field = parts
                    result = transformer.aggregate(path, op, field)
                    return str(round(result, 2) if isinstance(result, float) else result)
        
        # Select
        select_match = re.match(r'select\s+(\S+)\s+fields\s+(.+)', query)
        if select_match:
            path, fields_str = select_match.groups()
            fields = [f.strip() for f in fields_str.split(',')]
            result = transformer.select(path, fields)
            return json.dumps(result, indent=2)
        
        return "Unknown query format"
    
    except Exception as e:
        return f"Error: {str(e)}"


# Test the solution
print("=" * 60)
print("JSON TRANSFORMER SOLUTION")
print("=" * 60)

test_data = json.dumps({
    "users": [
        {"name": "Alice", "age": 30, "role": "engineer"},
        {"name": "Bob", "age": 25, "role": "designer"},
        {"name": "Charlie", "age": 35, "role": "engineer"},
        {"name": "Diana", "age": 28, "role": "manager"},
    ],
    "company": "TechCorp"
})

test_queries = [
    "get users.0.name",
    "filter users where age > 28",
    "count users",
    "avg users.age",
    "select users fields name,role",
]

for query in test_queries:
    print(f"\nQuery: {query}")
    print(f"Result: {transform_json(test_data, query)}")

---

## Challenge Solution: Tool Registry System

**Task**: Build a complete tool registry that can discover, validate, and manage tools dynamically.

In [ ]:
from typing import Callable, Dict, Any, List, Optional
from dataclasses import dataclass, field
from enum import Enum
import inspect
import json
import time
import math


class ToolCategory(Enum):
    MATH = "math"
    TEXT = "text"
    DATA = "data"
    WEB = "web"
    FILE = "file"
    UTILITY = "utility"


@dataclass
class ToolMetrics:
    """Usage metrics for a tool."""
    call_count: int = 0
    total_time: float = 0.0
    error_count: int = 0
    last_called: Optional[float] = None
    
    @property
    def avg_time(self) -> float:
        return self.total_time / self.call_count if self.call_count > 0 else 0.0
    
    @property
    def error_rate(self) -> float:
        return self.error_count / self.call_count if self.call_count > 0 else 0.0


@dataclass
class RegisteredTool:
    """A registered tool with metadata."""
    name: str
    description: str
    func: Callable
    category: ToolCategory
    parameters: Dict[str, str]  # param_name -> description
    examples: List[str] = field(default_factory=list)
    metrics: ToolMetrics = field(default_factory=ToolMetrics)
    enabled: bool = True


class ToolRegistry:
    """
    Production-ready tool registry with:
    - Dynamic tool registration
    - Automatic documentation
    - Usage tracking
    - Error handling
    - Tool discovery
    """
    
    def __init__(self):
        self._tools: Dict[str, RegisteredTool] = {}
        self._aliases: Dict[str, str] = {}  # alias -> tool_name
        self.call_history: List[Dict] = []
    
    def register(
        self,
        name: Optional[str] = None,
        category: ToolCategory = ToolCategory.UTILITY,
        description: Optional[str] = None,
        examples: Optional[List[str]] = None,
        aliases: Optional[List[str]] = None,
    ) -> Callable:
        """
        Decorator to register a function as a tool.
        
        Usage:
            @registry.register(category=ToolCategory.MATH)
            def calculate(expression: str) -> str:
                '''Calculate a math expression.'''
                ...
        """
        def decorator(func: Callable) -> Callable:
            tool_name = name or func.__name__
            tool_desc = description or func.__doc__ or "No description"
            
            # Extract parameters from function signature
            sig = inspect.signature(func)
            params = {
                p.name: str(p.annotation) if p.annotation != inspect.Parameter.empty else "Any"
                for p in sig.parameters.values()
            }
            
            # Create registered tool
            tool = RegisteredTool(
                name=tool_name,
                description=tool_desc.strip(),
                func=func,
                category=category,
                parameters=params,
                examples=examples or [],
            )
            
            self._tools[tool_name] = tool
            
            # Register aliases
            if aliases:
                for alias in aliases:
                    self._aliases[alias] = tool_name
            
            return func
        
        return decorator
    
    def call(self, name: str, **kwargs) -> Any:
        """Call a tool by name with arguments."""
        # Resolve alias
        tool_name = self._aliases.get(name, name)
        
        if tool_name not in self._tools:
            raise ValueError(f"Unknown tool: {name}")
        
        tool = self._tools[tool_name]
        
        if not tool.enabled:
            raise ValueError(f"Tool '{name}' is disabled")
        
        # Execute with timing
        start_time = time.time()
        error = None
        result = None
        
        try:
            result = tool.func(**kwargs)
        except Exception as e:
            error = str(e)
            tool.metrics.error_count += 1
            raise
        finally:
            elapsed = time.time() - start_time
            tool.metrics.call_count += 1
            tool.metrics.total_time += elapsed
            tool.metrics.last_called = time.time()
            
            # Log call
            self.call_history.append({
                "tool": tool_name,
                "args": kwargs,
                "result": str(result)[:100] if result else None,
                "error": error,
                "time": elapsed,
                "timestamp": time.time(),
            })
        
        return result
    
    def get_tool(self, name: str) -> Optional[RegisteredTool]:
        """Get tool by name or alias."""
        tool_name = self._aliases.get(name, name)
        return self._tools.get(tool_name)
    
    def list_tools(self, category: Optional[ToolCategory] = None) -> List[str]:
        """List all tool names, optionally filtered by category."""
        tools = self._tools.values()
        if category:
            tools = [t for t in tools if t.category == category]
        return [t.name for t in tools if t.enabled]
    
    def search(self, query: str) -> List[str]:
        """Search tools by name or description."""
        query = query.lower()
        results = []
        
        for tool in self._tools.values():
            if query in tool.name.lower() or query in tool.description.lower():
                results.append(tool.name)
        
        return results
    
    def get_documentation(self, name: Optional[str] = None) -> str:
        """Generate documentation for one or all tools."""
        if name:
            tool = self.get_tool(name)
            if not tool:
                return f"Tool not found: {name}"
            return self._format_tool_doc(tool)
        
        # Generate full documentation
        docs = ["# Tool Registry Documentation\n"]
        
        for category in ToolCategory:
            tools = [t for t in self._tools.values() if t.category == category]
            if tools:
                docs.append(f"\n## {category.value.title()} Tools\n")
                for tool in tools:
                    docs.append(self._format_tool_doc(tool))
        
        return "\n".join(docs)
    
    def _format_tool_doc(self, tool: RegisteredTool) -> str:
        """Format documentation for a single tool."""
        lines = [
            f"### {tool.name}",
            f"{tool.description}\n",
            "**Parameters:**",
        ]
        
        for param, ptype in tool.parameters.items():
            lines.append(f"- `{param}` ({ptype})")
        
        if tool.examples:
            lines.append("\n**Examples:**")
            for ex in tool.examples:
                lines.append(f"- {ex}")
        
        # Add metrics
        m = tool.metrics
        if m.call_count > 0:
            lines.append(f"\n**Stats:** {m.call_count} calls, {m.avg_time:.3f}s avg, {m.error_rate:.1%} errors")
        
        return "\n".join(lines)
    
    def get_stats(self) -> Dict[str, Any]:
        """Get registry statistics."""
        total_calls = sum(t.metrics.call_count for t in self._tools.values())
        total_errors = sum(t.metrics.error_count for t in self._tools.values())
        
        return {
            "total_tools": len(self._tools),
            "enabled_tools": len([t for t in self._tools.values() if t.enabled]),
            "total_calls": total_calls,
            "total_errors": total_errors,
            "error_rate": total_errors / total_calls if total_calls > 0 else 0,
            "tools_by_category": {
                cat.value: len([t for t in self._tools.values() if t.category == cat])
                for cat in ToolCategory
            },
        }
    
    def enable(self, name: str) -> None:
        """Enable a tool."""
        tool = self.get_tool(name)
        if tool:
            tool.enabled = True
    
    def disable(self, name: str) -> None:
        """Disable a tool."""
        tool = self.get_tool(name)
        if tool:
            tool.enabled = False


# Simple safe calculator using AST (consistent with lab teaching)
class SimpleSafeCalculator:
    """Safe calculator using AST parsing - no eval()!"""
    
    OPERATORS = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
    }
    
    FUNCTIONS = {
        'sqrt': math.sqrt,
        'abs': abs,
    }
    
    def evaluate(self, expression: str) -> float:
        """Safely evaluate a mathematical expression."""
        tree = ast.parse(expression, mode='eval')
        return self._eval_node(tree.body)
    
    def _eval_node(self, node) -> float:
        if isinstance(node, ast.Constant):
            return node.value
        elif isinstance(node, ast.BinOp):
            left = self._eval_node(node.left)
            right = self._eval_node(node.right)
            return self.OPERATORS[type(node.op)](left, right)
        elif isinstance(node, ast.UnaryOp):
            operand = self._eval_node(node.operand)
            return self.OPERATORS[type(node.op)](operand)
        elif isinstance(node, ast.Call):
            func_name = node.func.id if isinstance(node.func, ast.Name) else None
            if func_name in self.FUNCTIONS:
                args = [self._eval_node(arg) for arg in node.args]
                return self.FUNCTIONS[func_name](*args)
        raise ValueError(f"Unsupported expression: {ast.dump(node)}")


# Demonstration
print("=" * 60)
print("TOOL REGISTRY SOLUTION")
print("=" * 60)

# Create registry
registry = ToolRegistry()

# Register some tools - using SAFE calculator (no eval!)
@registry.register(
    category=ToolCategory.MATH,
    examples=["calculate('2 + 2')", "calculate('sqrt(16)')"]
)
def calculate(expression: str) -> str:
    """Safely evaluate a mathematical expression using AST parsing (no eval!)."""
    try:
        calc = SimpleSafeCalculator()
        result = calc.evaluate(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@registry.register(
    category=ToolCategory.TEXT,
    aliases=["upper", "uppercase"]
)
def to_uppercase(text: str) -> str:
    """Convert text to uppercase."""
    return text.upper()

@registry.register(category=ToolCategory.UTILITY)
def get_time() -> str:
    """Get current time."""
    return datetime.now().isoformat()

# Test registry
print("\n--- Calling Tools ---")
print(f"calculate('10 * 5'): {registry.call('calculate', expression='10 * 5')}")
print(f"upper('hello'): {registry.call('upper', text='hello')}")
print(f"get_time(): {registry.call('get_time')}")

print("\n--- Tool Search ---")
print(f"Search 'calc': {registry.search('calc')}")
print(f"Search 'time': {registry.search('time')}")

print("\n--- Registry Stats ---")
print(json.dumps(registry.get_stats(), indent=2))

print("\n--- Documentation ---")
print(registry.get_documentation('calculate'))

---

## Performance Comparison

Compare different implementation approaches.

In [ ]:
import time

def benchmark(func, *args, iterations=1000):
    """Benchmark a function."""
    start = time.perf_counter()
    for _ in range(iterations):
        func(*args)
    elapsed = time.perf_counter() - start
    return elapsed / iterations * 1000  # ms per call

# Benchmark unit converter
converter = UnitConverter()

print("Performance Benchmarks (ms per call):")
print(f"Unit conversion: {benchmark(converter.convert, 100, 'km', 'miles'):.4f}ms")

# Benchmark date calculator
calc = DateTimeCalculator()
print(f"Date difference: {benchmark(calc.date_difference, '2024-01-01', '2024-12-31'):.4f}ms")

# Benchmark JSON transformer
test_json = json.dumps({"users": [{"name": "Alice", "age": 30}] * 100})
transformer = JSONTransformer(test_json)
print(f"JSON filter (100 items): {benchmark(lambda: transformer.filter('users', 'age > 25')):.4f}ms")

---

## Key Takeaways

1. **Safety First**: Always validate inputs and avoid eval()
2. **Clear Interfaces**: Tools should have obvious input/output contracts
3. **Error Messages**: Provide helpful error messages that guide users
4. **Metrics**: Track usage to understand tool effectiveness
5. **Documentation**: Self-documenting tools are easier to use
6. **Testing**: Test edge cases and error conditions